In [ ]:
# import tensorflow as tf
# tf.__version__
# print(tf.config.list_physical_devices('GPU'))

2024-12-21 11:21:32.744506: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-21 11:21:32.777988: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-21 11:21:32.778031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-21 11:21:32.779580: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-21 11:21:32.787651: I tensorflow/core/platform/cpu_feature_guar

'2.15.0'

In [3]:
from ChemicalDice import smiles_preprocess, bioactivity, chemberta, Grover, ImageMol, chemical, quantum
import os
import pandas as pd

In [4]:
directory1 = "positive_Chemicaldice_data"
directory2 = "negative_Chemicaldice_data"
try:
    os.mkdir(directory1)
    os.mkdir(directory2)
    print(f"Directory '{directory1}' created successfully.")
    print(f"Directory '{directory2}' created successfully.")
except FileExistsError:
    print(f"Directory '{directory1}' already exists.")
    print(f"Directory '{directory2}' already exists.")

Directory 'positive_Chemicaldice_data' already exists.
Directory 'negative_Chemicaldice_data' already exists.


In [6]:
df = pd.read_csv('original_train.csv')
df.head()

,SMILES,ACTIVITY
0,Nc1nnc(o1)-c1ccc(o1)[N+](=O)[O-],1
1,O[C@H]1COC[C@@H]2O[C@H](CC[C@H]2N(C1)C(=O)Nc1c...,1
2,CC(C)C[C@@H](N)C(=O)N[C@@H]1[C@H](O)c2ccc(c(c2...,1
3,[O-][N+](=O)c1ccc(o1)/C=N/N1CC(=O)NC1=O,1
4,Cn1cnc(c1)CCNC(=O)C[C@@H]1CC[C@@H]2[C@H](COC[C...,1


In [8]:
positive_molecules = df[df['ACTIVITY'] == 1]
# positive_molecules.head()
print(len(positive_molecules))
# positive_molecules.to_csv('positive_molecules.csv', index=False)
# positive_molecules.head()

512


In [9]:
negative_molecules = df[df['ACTIVITY'] == 0]
print(len(negative_molecules))
negative_test_molecules = negative_molecules[:10]
negative_test_molecules.to_csv('negative_test_molecules.csv', index=False)

38800


### For positive molecules

In [10]:
# making a test csv(with 10 molecules) to check if pipeline runs
test_df = positive_molecules[:10]
# print(test_df)
test_df.to_csv('positive_test_molecules.csv', index=False)


In [11]:
# download prerequisites for quantum, grover and ImageMol
quantum.get_mopac_prerequisites()

Mopac is downloaded
Morse is compiled


In [12]:
# input file containing SMILES and labels
input_file = "positive_test_molecules.csv"
# preprocessing of smiles to different formats
smiles_preprocess.add_canonical_smiles(input_file)
print('done1')
smiles_preprocess.create_mol2_files(input_file, 'positive_temp_data/mol2files')
print('done2')
smiles_preprocess.create_sdf_files(input_file, 'positive_temp_data/sdffiles')
print('done3')

done1


  0%|          | 0/10 [00:00<?, ?it/s]==============================
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 2 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 58 62
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 1 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 33
100%|██████████| 10/10 [01:22<00:00,  8.27s/it]


done2
making directory  positive_temp_data/sdffiles


10it [00:00, 627.69it/s]

done3


In [15]:
# calculation of all descriptors
quantum.descriptor_calculator(input_file, output_file='positive_Chemicaldice_data/mopac.csv', output_dir='positive_temp_data/mopfiles')




          MOPAC Job: "positive_temp_data/mopfiles/C0.mop" ended normally on Dec 26, 2024, at 15:24.

Process 468651 found


          Thu Dec 26 15:24:36 2024  Job: 'positive_temp_data/mopfiles/C1' started successfully




          MOPAC Job: "positive_temp_data/mopfiles/C1.mop" ended normally on Dec 26, 2024, at 15:24.

Process 468660 found


          Thu Dec 26 15:24:52 2024  Job: 'positive_temp_data/mopfiles/C2' started successfully




          MOPAC Job: "positive_temp_data/mopfiles/C2.mop" ended normally on Dec 26, 2024, at 15:32.

Process 468669 found


          MOPAC Job: "positive_temp_data/mopfiles/C3.mop" ended normally on Dec 26, 2024, at 15:32.

Process 468734 found


...



          MOPAC Job: "positive_temp_data/mopfiles/C5.mop" ended normally on Dec 26, 2024, at 15:32.

Process 468785 found


          MOPAC Job: "positive_temp_data/mopfiles/C6.mop" ended normally on Dec 26, 2024, at 15:33.

Process 468797 found


          Thu Dec 26 15:33:09 2024  Job: 'positive_temp_data/mopfiles/C7' started successfully




          MOPAC Job: "positive_temp_data/mopfiles/C7.mop" ended normally on Dec 26, 2024, at 15:48.

Process 468552 found


          MOPAC Job: "positive_temp_data/mopfiles/C7.mop" ended normally on Dec 26, 2024, at 16:00.

Process 468633 found


          MOPAC Job: "positive_temp_data/mopfiles/C7.mop" ended normally on Dec 26, 2024, at 16:10.

Process 468807 found


          Thu Dec 26 16:10:54 2024  Job: 'positive_temp_data/mopfiles/C8' started successfully




          MOPAC Job: "positive_temp_data/mopfiles/C8.mop" ended normally on Dec 26, 2024, at 16:11.

Process 468969 found


          MOPAC Job: "positive_temp_data/mopfiles/C9.mop" ended normally on Dec 26, 2024, at 16:11.

Process 468979 found



In [16]:
Grover.get_embeddings(input_file,  output_file_name="positive_Chemicaldice_data/Grover.csv", output_dir='positive_temp_data')

Grover model is downloaded
Grover model is extracted


100%|██████████| 10/10 [00:00<00:00, 17.24it/s]
Total size = 10
Generating...


Loading data


Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_q.act_func.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_q.W_h.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_k.act_func.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_k.W_h.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_v.act_func.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.0.mpn_v.W_h.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.1.mpn_q.act_func.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.1.mpn_q.W_h.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.1.mpn_k.act_func.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.1.mpn_k.W_h.weight".
Loading pretrained parameter "grover.encoders.edge_blocks.0.heads.1.mpn_v.act_func.weight".
Loading pretr

In [17]:
ImageMol.image_to_embeddings(input_file, output_file_name='positive_Chemicaldice_data/ImageMol.csv', output_dir='positive_temp_data' )

making directory  positive_temp_data/images
ImageMol model is downloaded


/home/suvenduk/.conda/envs/chemicaldice/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/suvenduk/.conda/envs/chemicaldice/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [18]:
chemberta.smiles_to_embeddings(input_file, output_file = "positive_Chemicaldice_data/Chemberta.csv")

100%|██████████| 10/10 [00:00<00:00, 44.68it/s]


In [20]:
# san_check = pd.read_csv('test_molecules.csv')
# print(True in san_check['Canonical_SMILES'].isna())

In [24]:
# import tensorflow as tf
# tf.test.is_gpu_available()

In [23]:
bioactivity.calculate_descriptors(input_file, output_file = "positive_Chemicaldice_data/Signaturizer.csv")

Parsing SMILES: 10it [00:00, 3513.99it/s]
Generating signatures:   0%|          | 0/1 [00:00<?, ?it/s]2024-12-26 17:04:57.532611: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 0s 356ms/step


Generating signatures: 100%|██████████| 1/1 [00:01<00:00,  1.74s/it]

Descictors saved to  positive_Chemicaldice_data/Signaturizer.csv


In [26]:
chemical.descriptor_calculator(input_file, output_file="positive_Chemicaldice_data/mordred.csv")

0it [00:00, ?it/s]

2it [00:00,  2.97it/s]/home/suvenduk/.conda/envs/chemicaldice/lib/python3.9/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
10it [00:07,  1.34it/s]


In [27]:
from ChemicalDice.fusionData import fusionData
data_paths = {
   "Chemberta":"positive_Chemicaldice_data/Chemberta.csv",
   "Grover":"positive_Chemicaldice_data/Grover.csv",
   "mopac":"positive_Chemicaldice_data/mopac.csv",
   "mordred":"positive_Chemicaldice_data/mordred.csv",
   "Signaturizer":"positive_Chemicaldice_data/Signaturizer.csv",
   "ImageMol": "positive_Chemicaldice_data/ImageMol.csv"
}

In [28]:
fusiondata = fusionData(data_paths = data_paths, label_file_path="positive_test_molecules.csv", label_column="ACTIVITY", id_column="id")

Successfully loaded, processed for 'Chemberta'.
Successfully loaded, processed for 'Grover'.
Successfully loaded, processed for 'mopac'.
Successfully loaded, processed for 'mordred'.
Successfully loaded, processed for 'Signaturizer'.
Successfully loaded, processed for 'ImageMol'.


In [29]:
fusiondata.dataframes

{'Chemberta':     ChB77MLM_0  ChB77MLM_1  ChB77MLM_2  ChB77MLM_3  ChB77MLM_4  ChB77MLM_5  \
 id                                                                           
 C0   -0.012744    0.224102    0.085117   -0.061324   -0.086688    0.025257   
 C1    0.067436    0.076087    0.043730    0.068862    0.013719    0.081564   
 C2    0.121448    0.047547    0.010370    0.070014   -0.014906   -0.005577   
 C3    0.133229    0.179282   -0.012287    0.065493   -0.030343   -0.101416   
 C4    0.066044    0.059281    0.052327    0.066178    0.000116    0.099292   
 C5    0.093008    0.046539    0.031208    0.072988    0.021473    0.089307   
 C6    0.165586    0.260226   -0.032918   -0.010465   -0.049788    0.022497   
 C7    0.088290    0.074341   -0.139458    0.008644    0.005894   -0.014828   
 C8   -0.006272    0.037161   -0.041803    0.112964    0.056105    0.106620   
 C9    0.110842    0.060920   -0.008159    0.014547   -0.019299   -0.013614   
 
     ChB77MLM_6  ChB77MLM_7  ChB77MLM

In [30]:
fusiondata.keep_common_samples()

In [31]:
fusiondata.ShowMissingValues()
fusiondata.remove_empty_features(threshold=20)
fusiondata.ShowMissingValues()

Dataframe name: Chemberta
Missing values: 0

Dataframe name: Grover
Missing values: 0

Dataframe name: mopac
Missing values: 0

Dataframe name: mordred
Missing values: 2904

Dataframe name: Signaturizer
Missing values: 0

Dataframe name: ImageMol
Missing values: 0



KeyError: "['SM1_D', 'SM1_A'] not in index"